# Table of Contents
 <p><div class="lev2 toc-item"><a href="#Defaultgen-function" data-toc-modified-id="Defaultgen-function-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Defaultgen function</a></div><div class="lev2 toc-item"><a href="#Predict-function" data-toc-modified-id="Predict-function-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Predict function</a></div><div class="lev2 toc-item"><a href="#Find-Match-indices--function" data-toc-modified-id="Find-Match-indices--function-03"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Find Match indices  function</a></div><div class="lev2 toc-item"><a href="#conf_loss-function" data-toc-modified-id="conf_loss-function-04"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>conf_loss function</a></div>

## Defaultgen function 

Input parameters:  original input image, Feature map, width of default_box, height of default_box 
Output parameters : list of default boxes 

In [3]:
import numpy as np 

Input_image  = np.ones((8,8))
Feature_map = np.ones((4, 4))
Width=  10 
Height = 10 


In [4]:
def defaultgen( Input_image, Feature_map, Width, Height): 
    
    
    #number of boxes is the number of pixels in feature map. Default box for each pixel.  
    boxes = np.zeros((Feature_map.shape[0], Feature_map.shape[1], 6))
    
    for i in range(Feature_map.shape[0]): 
        for j in range(Feature_map.shape[1]): 
            
            # d depends on the number of max pooling layers
          
            scale = int(Input_image.shape[0]/ Feature_map.shape[0]) 
            cx  = int((( scale- 1)/ 2 )+ (j*(scale )))
            cy  = int(((scale - 1)/ 2 )+ (i*(scale ))) 
            
            boxes[i,j,:] = np.array([cx,cy,Width, Height, 0, 0])
            
    return boxes
    

In [5]:
boxes = defaultgen( Input_image, Feature_map, Width, Height)
boxes2 = boxes.reshape([boxes.shape[0]*boxes.shape[1],6])
boxes2.shape

(16, 6)

## Predict function 

In [6]:
Bshift= np.zeros((Feature_map.shape[0]*Feature_map.shape[0], 6))
Bshift[:,0:4]= 1


def predict(Bshift, B): 
    
    cx_pred = Bshift[:,0]*B[:,2] + B[:,0]
    cy_pred = Bshift[:,1]*B[:,2] + B[:,1]
    W_pred = np.exp(Bshift[:,2])*B[:,2]
    H_pred = np.exp(Bshift[:,3])*B[:,3]
    class1 = Bshift[:,4]
    class2 = Bshift[:,5]
    Predicted_boxes = np.vstack((cx_pred,cy_pred,W_pred,H_pred, class1, class2))
    Predicted_boxes = np.transpose(Predicted_boxes)
    
    return Predicted_boxes



In [7]:
predicted_boxes = predict(Bshift, boxes2)

In [8]:
predicted_boxes

array([[ 10.        ,  10.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 12.        ,  10.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 14.        ,  10.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 16.        ,  10.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 10.        ,  12.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 12.        ,  12.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 14.        ,  12.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 16.        ,  12.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 10.        ,  14.        ,  27.18281828,  27.18281828,
          0.        ,   0.        ],
       [ 12.        ,  14.        ,  27.18281828,  27.18281828,
          0.        ,   0. 

In [89]:
boxes2[0]

array([  0.,   0.,  10.,  10.,   0.,   0.])

In [90]:
Bshift[0]

array([ 1.,  1.,  1.,  1.,  0.,  0.])

## Find Match indices  function 


In [465]:
def bb_intersection_over_union(boxA, boxB):
        # determine the (x, y)-coordinates of the intersection rectangle
        
    boxA = [boxA[0],boxA[1], boxA[0]+boxA[2],boxA[1]+boxA[3]  ]
    boxB = [boxB[0],boxB[1], boxB[0]+boxB[2],boxB[1]+boxB[3]  ]
    
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # compute the area of intersection rectangle
    interArea = (xB - xA ) * (yB - yA )
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxa_area = (boxA[2] - boxA[0] ) * (boxA[3] - boxA[1] )
    boxb_area = (boxB[2] - boxB[0] ) * (boxB[3] - boxB[1] )
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxa_area + boxb_area - interArea)

    # return the intersection over union value
    return iou

In [466]:
gt = np.array(([2,2,3,3,1,0],[3,3,3,3,1,0]))

# pred_boxes= np.array([[2,2,1,1,0,1],[2,2,1,2,0,1],[2,2,3,3,1,0 ],[3,3,3,3,1,0 ],[1,1,3,3,1,0 ],[4,4,3,3,1,0 ],[1,1,1,1,1,0 ]   ] )
pred_boxes= np.array([[2,2,1,1,0,1],[2,2,1,2,0,1],[2,2,3,3,1,0 ]  ] )

def find_match_indices(gt, pred_boxes): 
    
    pos_iou_score =  np.zeros((pred_boxes.shape[0]*gt.shape[0],3 ))
    neg_iou_score = np.zeros((pred_boxes.shape[0]*gt.shape[0], 3 ))
    
    iterator =0
    
    for i in range(gt.shape[0]): 
        for j in range(pred_boxes.shape[0]): 
            iou_score  = bb_intersection_over_union(gt[i,0:4],pred_boxes[j,0:4] )
            print(i,j, iou_score)
        
            if iou_score > 0.5:  
                pos_iou_score[iterator,:] =i,j,iou_score
                neg_iou_score[iterator,:] = [i,j,-1]
            else :
                neg_iou_score[iterator,:] = [i,j,iou_score]
            
            iterator+=1
    pos_inds  = pos_iou_score[np.where(pos_iou_score[:,2]>0),0:2]
    neg_samples = pos_inds.shape[0]*3 
    
    neg_iou_score2 = neg_iou_score[np.where(neg_iou_score[:,2]>-1),:]
    neg_iou_score2 =neg_iou_score2[0]
    
    neg_iou_score2 = neg_iou_score2[neg_iou_score2[:,2].argsort()]
    neg_iou_score2 = neg_iou_score2[neg_iou_score2.shape[0]-neg_samples:neg_iou_score2.shape[0], :]
    neg_inds = neg_iou_score2[:,0:2]
    return  pos_inds, neg_inds, pos_iou_score, neg_iou_score2

In [467]:
pos_inds, neg_inds, pos_iou_score, neg_iou_score= find_match_indices(gt, pred_boxes)

0 0 0.111111111111
0 1 0.222222222222
0 2 1.0
1 0 0.0
1 1 0.0
1 2 0.285714285714


In [468]:
pos_inds

array([[[ 0.,  2.]]])

In [469]:
neg_inds

array([[ 0.,  0.],
       [ 0.,  1.],
       [ 1.,  2.]])

## conf_loss function

class loss where the loss is calculated between the predicted and the ground truth boxes. Loss if first calculated for the positive indices and then for the negative indices 

In [541]:
# predicted boxes classes
dbclass = pred_boxes[:,4:6]

#ground truth boxes classes
gtclass = gt[:,4:6]

#exponential of the predicted classes
exp_dbclass=np.exp(dbclass) 

#sum of the exponential 
sum_exp_dbclass = np.sum(np.exp(dbclass), axis=1)

db_probs =np.array([exp_dbclass[:,0]/sum_exp_dbclass,exp_dbclass[:,1]/sum_exp_dbclass] )
db_probs = db_probs.transpose()

pos_loss= 0 

for i in range(pos_inds.shape[0]):
    
    p_ind = pos_inds[i,:]
    p_ind = p_ind[0].astype(np.uint8)
    gt_prob = gtclass[p_ind[0], : ]
    db_prob = db_probs[p_ind[1], : ]
    pos_loss+= np.dot(gt_prob, -np.log(db_prob))

neg_loss= 0     
for i in range(neg_inds.shape[0]):
    p_ind = neg_inds[i,:]
    p_ind = p_ind.astype(np.uint8)
    gt_prob = gtclass[p_ind[0], : ]
    db_prob = db_probs[p_ind[1], : ]
    neg_loss+= np.dot(gt_prob, -np.log(1-db_prob))


ValueError: operands could not be broadcast together with shapes (2,) (3,) 

In [540]:
neg_loss

1.9397850625546684

In [520]:
pos_inds[0][0][0]


0.0